In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import math
import pickle
import Niklas_Prepare_Data as npd

In [2]:
# Prepare stirps to use them later
npd.prepare_all_data(False)

Finish strip_1_train
Finish strip_2_train
Finish strip_3_train
Finish strip_4_train


In [36]:
def load_model(x: int):
    with open("./model/model_%i.bin" % x, 'rb') as file:
        clf = pickle.load(file)
    return clf

In [37]:
load_model(2) 

RandomForestClassifier(n_estimators=200, random_state=0)

In [40]:
output_dir = '../data/fixed_train1/'

In [93]:
def load_strip_pkl(path: str)-> pd.DataFrame: 
    return pd.read_pickle(path)

def save_strip_pkl(data: pd.DataFrame, outdir: str, outname: str):
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)    

    data.to_pickle(fullname)

def load_strip_csv(path: str)-> pd.DataFrame: 
    return pd.read_csv(path, sep=',')

def save_strip_csv(data: pd.DataFrame, outdir: str, outname: str):
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)    

    data.to_csv(fullname, sep=',')
# format table to use it for classifier
def split_data(df: pd.DataFrame) -> pd.DataFrame:
    columns = ["ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "r"] # "strip_id", "timestamp", 
    newColumns = range(1,16)
    df2 = pd.DataFrame()
    for c in columns:
        cols = list(map(lambda x: c+str(x), newColumns))
        new_df = pd.DataFrame(df[c].to_list(), columns=cols)
        df2 = pd.concat([df2, new_df], axis=1)
    # add near column 
    df2['vicon_x'] = pd.DataFrame(df['vicon_x'].values.tolist()).agg('max', axis=1)
    df2['vicon_y'] = pd.DataFrame(df['vicon_y'].values.tolist()).agg('max', axis=1)
    df2['near'] = pd.DataFrame(df['near'].values.tolist()).agg('max', axis=1)
    return df2

def prepare_training_data(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    df = split_data(df)
    #Alle frames entfernen, wo near = 0
    df = df[df.near != 0]

    X_train = df.drop('vicon_x',axis = 1).drop('vicon_y',axis = 1).drop('near',axis = 1)
    X_train.fillna(X_train.mean(), inplace=True)
    Y_train = df[['vicon_x', 'vicon_y']]
    Y_train.fillna(Y_train.mean(), inplace=True)
    
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_transformed = sc.transform(X_train)

    return X_train_transformed, Y_train

In [98]:
# Read tranformed data
for i in range(2, 3):
    df = load_strip_pkl(output_dir+'strip_%i_train.pkl' % i)
    X_train, Y_train = prepare_training_data(df)

    X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X_train, Y_train, test_size=0.05)   

    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    
    forest = RandomForestRegressor(n_estimators=200, random_state = 0)
    forest.fit(X_train1, Y_train1)

    prediction = forest.predict(X_test1)
            
    score = mean_squared_error(Y_test1, prediction)
    print("mse " + str(score))

mse 0.03341118833105259
